En este [ejemplo](https://towardsdatascience.com/building-a-logistic-regression-in-python-step-by-step-becd4d56c9c8), emplearemos la base de datos 'banking.csv'. El conjunto de datos proporciona la información de los clientes bancarios. Incluye 41.188 registros y 21 campos.

El conjunto de datos proviene del repositorio [UCI Machine Learning](http://archive.ics.uci.edu/ml/about.html) y está relacionado con campañas de marketing directo (llamadas telefónicas) de una institución bancaria portuguesa.

El ***objetivo*** de la clasificación es ***predecir si el cliente suscribirá un depósito a largo plazo***.

**Variables de entrada**
* **age** (numérica): edad del cliente
* **job** (categórica): profesión o tipo de trabajo (“admin”, “blue-collar”, “entrepreneur”, “housemaid”, “management”, “retired”, “self-employed”, “services”, “student”, “technician”, “unemployed”, “unknown”)
* **marital** (categórica): estado civil (“divorced”, “married”, “single”, “unknown”)
* **education** (categórica): nivel educativo (“basic.4y”, “basic.6y”, “basic.9y”, “high.school”, “illiterate”, “professional.course”, “university.degree”, “unknown”)
* **default** (categórica): existencia de impagos (“no”, “yes”, “unknown”)
* **housing** (categórica): existencia de préstamo de vivienda (“no”, “yes”, “unknown”)
* **loan** (categórica): existencia de préstamo personal (“no”, “yes”, “unknown”)
* **contact** (categórica): tipo de comunicación del contacto (“cellular”, “telephone”)
* **month** (categórica): último contacto mes del año (“jan”, “feb”, “mar”, …, “nov”, “de)
* **day_of_week** (categórica): último día de contacto de la semana (“mon”, “tue”, “wed”, “thu”, “fri”)
* **duration** (numérica): duración del último contacto, en segundos. Nota importante: este atributo afecta en gran medida el objetivo de salida (por ejemplo, si la duración = 0 entonces y = 'no'). La duración no se conoce antes de que se realice una llamada, también, después del final de la llamada, obviamente se conoce y. Por lo tanto, esta entrada solo debe incluirse con fines de referencia y debe descartarse si la intención es tener un modelo predictivo realista.
* **campaign** (numérica): número de contactos realizados durante esta campaña y para este cliente
* **pdays** (numérica): número de días que pasaron después de que el cliente fue contactado por última vez de una campaña anterior (999 significa que el cliente no fue contactado previamente)
* **previous** (numérica): número de contactos realizados antes de esta campaña y para este cliente
* **poutcome** (categórica): resultado de la campaña de marketing anterior (“failure”, “nonexistent”, “success”)
* **emp.var.rate** (numérica): tasa de variación del empleo
* **cons.price.idx** (numérica): índice de precios al consumidor
* **cons.conf.idx** (numérica): índice de confianza del consumidor
* **euribor3m** (numérica): tipo euribor a 3 meses
* **nr.employed** (numérica): número de empleados

**Variable outcome:**

*   **y** (dicotómica)— ¿el cliente ha suscrito un depósito a plazo? (“1” significa “Sí”, “0” significa “No”).

**Librerías:**

Emplearemos **`panda`** (manipulación y análisis de datos), **`numpy`** (manipulación de arrays), **`scikit-learn`** (análisis predictivo para machine learning), **`matplotlib`** (visualización de datos) y **`seaborn`** (visualización de datos, basada en matplotlib).

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

In [3]:
from pandas import read_csv
datos=read_csv('AED_banking.csv', header=0)
datos.head()

FileNotFoundError: [Errno 2] No such file or directory: 'AED_banking.csv'

Aunque omitimos los datos ausentes (NA) con la función `dropna()`, comprobamos que no hay. Finalmente, comprobamos que tenemos 41188 observaciones de 21 variables.

In [ ]:
datos=datos.dropna()
print(datos.shape)
print(list(datos.columns))

**Preparación de los datos**

La columna de `education` del conjunto de datos tiene muchas categorías y necesitamos reducirlas para un mejor modelado. La columna de educación tiene las siguientes categorías:

In [ ]:
datos['education'].unique()

Agruparemos los niveles "basic.4y", "basic.6y", y "basic.9y" en el nuevo nivel "basic"

In [ ]:
datos['education']=np.where(datos['education'] =='basic.9y', 'basic', datos['education'])
datos['education']=np.where(datos['education'] =='basic.6y', 'basic', datos['education'])
datos['education']=np.where(datos['education'] =='basic.4y', 'basic', datos['education'])

In [ ]:
datos['education'].unique()

**Exploración de datos**

In [ ]:
datos['y'].value_counts()

Comprobamos que la mayor parte de clientes no disponen de crédito a largo plazo (36548/36548+4640), mientras que algo más de un 10% sí que disponen (4640/4640+36548).

In [ ]:
sns.countplot(x='y', data=datos, palette='hls')
plt.show()

Calculemos el porcentaje de clientes que disponen de crédito a largo plazo frente a aquellos que no están suscritos a un crédito a largo plazo.

In [ ]:
count_no_sub = len(datos[datos['y']==0])
count_sub = len(datos[datos['y']==1])
pct_of_no_sub = count_no_sub/(count_no_sub+count_sub)
print("El porcentaje de no suscripción de crétito a largo plazo es", pct_of_no_sub*100,"%")
pct_of_sub = count_sub/(count_no_sub+count_sub)
print("El porcentaje de suscripción de crétito a largo plazo es", pct_of_sub*100,"%")

Comprobamos que los niveles o clases de la VD están desequilibrados, y la proporción de instancias sin suscripción a suscripción es 89:11. Antes de seguir adelante para equilibrar las clases, exploremos un poco más las características del resto de variables cuantitativas en función de los niveles de la VD.

In [ ]:
datos.groupby('y').mean()

**Observaciones:**
* La edad media de los clientes que compraron el depósito a plazo es ligeramente superior a la de los clientes que no lo hicieron.
* La duración del último contacto es muy superior en los suscriptores al crédito.
* Sorprendentemente, las campañas (número de contactos o llamadas realizadas durante la campaña actual) son menores para los clientes que compraron el depósito a plazo.
* Los pdays (días desde la última vez que se contactó al cliente) son comprensiblemente más bajos para los clientes que lo compraron. Cuanto más bajos sean los pdays, mejor será el recuerdo de la última llamada y, por lo tanto, mayores posibilidades de venta.

Podemos calcular medias categóricas para otras variables categóricas, como la educación y el estado civil, para obtener una idea más detallada de nuestros datos.

In [ ]:
datos.groupby('job').mean()

In [ ]:
datos.groupby('marital').mean()

In [ ]:
datos.groupby('education').mean()

**Visualizaciones**

Procedemos a visualizar el comportamiento de las variables predictoras categóricas en función de los niveles de la VD. Esto nos permite comprobar visualmente si un predictor determinado puede ser útil a la hora de predecir la probabilidad del suceso.

In [ ]:
%matplotlib inline
pd.crosstab(datos.job,datos.y).plot(kind='bar')
plt.title('Frecuencia de compra en función de la profesión laboral')
plt.xlabel('Cargo')
plt.ylabel('Frecuencia de compra de crédito a largo plazo')

La frecuencia de compra del depósito depende en gran medida de la profesión. Por lo tanto, la profesión laboral podría ser un buen predictor de la variable resultado.

In [ ]:
table=pd.crosstab(datos.marital,datos.y)
table.div(table.sum(1).astype(float), axis=0).plot(kind='bar', stacked=True)
plt.title ('Gráfico de barras apiladas de educación frente a compras')
plt.xlabel('Educación')
plt.ylabel('Proporción de clientes')

El estado civil no parece un predictor de la variable resultado, dado que no parece diferenciar sus niveles.

In [ ]:
table=pd.crosstab(datos.education,datos.y)
table.div(table.sum(1).astype(float), axis=0).plot(kind='bar', stacked=True)
plt.title('Gráfico de barras apiladas de educación frente a compras')
plt.xlabel('Educación')
plt.ylabel('Proporción de clientes')

La educación parece un buen predictor de la variable de resultado.

In [ ]:
pd.crosstab(datos.day_of_week,datos.y).plot(kind='bar')
plt.title('Frecuencia de compra por día de la semana')
plt.xlabel('Día de la semana')
plt.ylabel('Frecuencia de compra')

El día de la semana puede no ser un buen predictor del resultado.

In [ ]:
pd.crosstab(datos.month,datos.y).plot(kind='bar')
plt.title('Frecuencia de compra por mes')
plt.xlabel('Mes')
plt.ylabel('Frecuencia de compra')

El mes podría ser un buen predictor de la variable de resultado.

In [ ]:
datos.age.hist()
plt.title('Histograma de edad')
plt.xlabel('Edad')
plt.ylabel('Frecuencia')

La mayoría de los clientes del banco en este conjunto de datos tienen entre 30 y 40 años.

In [ ]:
pd.crosstab(datos.poutcome,datos.y).plot(kind='bar')
plt.title('Frecuencia de compra por Poutcome')
plt.xlabel('Poutcome')
plt.ylabel('Frecuencia de compra')

Poutcome parece ser un buen predictor de la variable de resultado.

**Creación de variables ficticias**

Antes de crear el modelo de regresión logística, es necesario codificar como variables ficticias o dummies aquellas variables de tipo categórico. En nuestro dataset existen varias variables: la profesión (job), el estado civil (marital), el nivel educativo (education), la existencia de impagos (default), de préstamos de vivienda (housing), de préstamo personal (loan), el tipo de comunicación de contacto (contact), el mes (month), el día de la semana (day_of_week) y el resultado de la campaña anterior (poutcome).

In [ ]:
cat_vars=['job','marital','education','default','housing','loan','contact','month','day_of_week','poutcome']
for var in cat_vars:
    cat_list='var'+'_'+var
    cat_list = pd.get_dummies(datos[var], prefix=var)
    datos1=datos.join(cat_list)
    datos=datos1
cat_vars=['job','marital','education','default','housing','loan','contact','month','day_of_week','poutcome']
datos_vars=datos.columns.values.tolist()
to_keep=[i for i in datos_vars if i not in cat_vars]

In [ ]:
datos_final=datos[to_keep]
datos_final.columns.values

**Sobremuestreo usando SMOTE**

https://machinelearningmastery.com/smote-oversampling-for-imbalanced-classification/


In [ ]:
X = datos_final.loc[:, datos_final.columns != 'y']
y = datos_final.loc[:, datos_final.columns == 'y']

In [ ]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
columns = X_train.columns
os_data_X,os_data_y=oversample.fit_resample(X_train, y_train)

In [ ]:

os_data_X = pd.DataFrame(data=os_data_X,columns=columns )
os_data_y= pd.DataFrame(data=os_data_y,columns=['y'])
# podemos comprobar los números de nuestros datos
print("La longitud de los datos sobremuestreados es ",len(os_data_X))
print("Número de sin suscripción en datos sobremuestreados",len(os_data_y[os_data_y['y']==0]))
print("Número de suscripción",len(os_data_y[os_data_y['y']==1]))
print("La proporción de datos sin suscripción en datos sobremuestreados es ",len(os_data_y[os_data_y['y']==0])/ len(os_data_X))
print("La proporción de datos de suscripción en datos sobremuestreados es ",len(os_data_y[os_data_y['y']==1])/len(os_data_X))

Ahora tenemos datos perfectamente equilibrados. Es posible que haya notado que hice un sobremuestreo solo en los datos de entrenamiento, porque al hacer un sobremuestreo solo en los datos de entrenamiento, ninguna información en los datos de prueba se usa para crear observaciones sintéticas, por lo que ninguna información de los datos de prueba se filtrará en la formación del modelo.

**Eliminación recursiva de características**

La eliminación recursiva de características ([RFE](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html)) se basa en la idea de construir repetidamente un modelo y elegir la característica con mejor o peor rendimiento, dejandola a un lado y luego repitiendo el proceso con el resto de las predictores. Este proceso se aplica hasta que se agotan todas las entidades del conjunto de datos. El objetivo de RFE es seleccionar características considerando de forma recursiva conjuntos de características cada vez más pequeños.

In [ ]:
datos_final_vars=datos_final.columns.values.tolist()
y=['y']
X=[i for i in datos_final_vars if i not in y]
X

In [ ]:
import statsmodels.api as sm
logit_model=sm.Logit(y,X)
result=logit_model.fit()
print(result.summary2())

In [ ]:
cols=['marital_divorced', 'marital_married', 'marital_single', 'marital_unknown', 'education_basic', 'education_high.school', 'education_professional.course',
      'education_university.degree', 'education_unknown', 'housing_no', 'housing_unknown', 'housing_yes', 'loan_no', 'loan_unknown', 'loan_yes',
      'day_of_week_fri', 'day_of_week_mon', 'day_of_week_thu', "day_of_week_tue", "day_of_week_wed"]
X=os_data_X[cols]
y=os_data_y['y']

Implementación del modelo con Statmodels

In [ ]:
import statsmodels.api as sm
logit_model=sm.Logit(y,X)
result=logit_model.fit()
print(result.summary2())